<a href="https://colab.research.google.com/github/AhmedSSoliman/Llama2-CodeGen-Fine-Tuning-LLama-2/blob/master/Llama2_CodeGen_Fine_Tuning_LLama_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



## Finetune Llama-2-7b on a Google colab

This notebook shows how to fine-tune the Llama-2-7b model on Google colab using AutoTrain.

We will leverage PEFT library from Hugging Face ecosystem, as well as QLoRA for more memory efficient finetuning

# **Install Dependencies**

Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent [`SFTTrainer`](https://huggingface.co/docs/trl/main/en/sft_trainer). We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install `einops` as it is a requirement to load Falcon models.

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 

[codecarbon INFO @ 13:11:37] Energy consumed for RAM : 0.003020 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 13:11:37] Energy consumed for all GPUs : 0.009861 kWh. Total GPU Power : 31.445 W
[codecarbon INFO @ 13:11:37] Energy consumed for all CPUs : 0.013457 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:11:37] 0.026337 kWh of electricity used since the beginning.



Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


# **Dataset**

In [ ]:
from datasets import load_dataset

dataset_name = "kejian/codesearchnet-python-raw-457k"
dataset = load_dataset(dataset_name)




In [ ]:
dataset = dataset.remove_columns(['repo', 'path', 'url', 'code_tokens', 'docstring_tokens', 'language', 'partition', 'avg_line_len'])

In [ ]:
# Convert the dataset to a Pandas DataFrame

import pandas as pd
df = pd.DataFrame(dataset)


#read csv file
#df = pd.read_csv('train.csv')

Comment removal from the docstring column

In [ ]:
import re

#regex for matching any comments starts and ends with """
pattern1='(?s)(""")(.*?)(""")'
#regex for matching any comments starts and ends with '''
pattern2= '(?s)(\''')(.*?)(\''')'
#regex for matching any comments starts with #
pattern3 = r'(#.*)'
#iterate over all rows in df
for i in range(0,len(df)):
    print('process row# : ',i)
    row=df.iloc[i]
    code=row[0]
    code=re.sub(pattern1, '', code)
    code=re.sub(pattern2, '', code)
    code=re.sub(pattern3, '', code)
    row[0]=code

df.to_csv('trains_clean.csv')



In [ ]:
from datasets import load_dataset

# Step 1: Load CSV file as `DatasetDict`
csv_dataset = load_dataset("csv", data_files="train_clean.csv")

# Step 2: Push `DatasetDict` object to HuggingFace Hub
csv_dataset.push_to_hub("AhmedSSoliman/CodeSearchNet")

# **Configure Dataset for training**

In [ ]:
from datasets import load_dataset

dataset_name = "AhmedSSoliman/CodeSearchNet"
dataset = load_dataset(dataset_name)

  0%|          | 0/3 [00:00<?, ?it/s]

Select only 2000 rows random from the dataset

In [ ]:
import random
train_dataset = dataset['train']
# Get the total number of records in the dataset
num_records = len(train_dataset)

# Generate a list of 100k random indices
indices = random.sample(range(num_records), 2000)

# Select the 100k records with the random indices
train_dataset = train_dataset.select(indices)
dataset = train_dataset

In [ ]:
dataset

Dataset({
    features: ['code', 'docstring'],
    num_rows: 2000
})

In [ ]:
import pandas as pd
# Convert the dataset to a Pandas DataFrame
df = pd.DataFrame(dataset)

In [ ]:
df.head()

,code,docstring
0,def parse_hh_mm_ss(self):\n \n s...,Parses raw time\n\n :return: Time parsed
1,def specificity(self):\n \n ids ...,Loosely based on http://www.w3.org/TR/REC-CSS2...
2,def fw_int_to_hex(*args):\n \n return bi...,Pack integers into hex string.\n\n Use litt...
3,"def tryCKeywords(self, block, isBrace):\n ...","Check for if, else, while, do, switch, private..."
4,def local_obj(self):\n \n if sel...,Buffered result of :meth:`make` which is (prob...


Add text column to the dataset which contains the Question and the answer of this quesion through the dataset

In [ ]:
text_col = []

for _, row in df.iterrows():

    input_query = str(row['docstring'])
    response = str(row['code'])

    if(len(input_query.strip())==0):
      text = "### Input: \n" + input_query + "\n### Response: \n" + response

    else:
      text =  (
           "### Input: \n"
          + input_query
          + "\n### Response: \n"
          + response
      )

      text_col.append(text)


In [ ]:
df.loc[:, "text"] = text_col
print(df.head)

<bound method NDFrame.head of                                                    code  \
0     def parse_hh_mm_ss(self):\n        \n        s...   
1     def specificity(self):\n        \n        ids ...   
2     def fw_int_to_hex(*args):\n    \n    return bi...   
3     def tryCKeywords(self, block, isBrace):\n     ...   
4     def local_obj(self):\n        \n        if sel...   
...                                                 ...   
1995  def notify_created(room, event, user):\n    \n...   
1996  def julian(mon, day, year):\n    \n    ig = 15...   
1997  def processMailList(platformNames=[], emails=[...   
1998  def push_progress(self, status, object_id, pro...   
1999  def unauthorized_callback(self):\n        \n  ...   

                                              docstring  \
0       Parses raw time\n\n        :return: Time parsed   
1     Loosely based on http://www.w3.org/TR/REC-CSS2...   
2     Pack integers into hex string.\n\n    Use litt...   
3     Check for if, else,

In [ ]:
df.to_csv("train.csv", index = False)


Pushing the CSV file data to the huggingface hub

In [ ]:
from datasets import load_dataset

# Step 1: Load CSV file as `DatasetDict`
csv_dataset = load_dataset("csv", data_files="train.csv")

# Step 2: Push `DatasetDict` object to HuggingFace Hub
csv_dataset.push_to_hub("AhmedSSoliman/CodeSearchNet-py")


This is another function to add a text column which contains the question and the corresponding answer

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['code'])):
        text = f"### Question: {example['docstring'][i]}\n ### Answer: {example['code'][i]}"
        output_texts.append(text)
    return output_texts



# **Autotrain**

In [ ]:
!pip install autotrain-advanced

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 63.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 31.0 MB/s eta

In [ ]:
!autotrain setup

2023-07-26 11:49:20.699683: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-26 11:49:22.960 | WARNING  | autotrain.cli.run_dreambooth:<module>:15 - ❌ Some DreamBooth components are missing! Please run `autotrain setup` to install it. Ignore this warning if you are not using DreamBooth or running `autotrain setup` already.
2023-07-26 11:49:23.848 | INFO     | autotrain.cli.run_setup:run:34 - Installing latest transformers@main
2023-07-26 11:49:52.936 | INFO     | autotrain.cli.run_setup:run:36 - Successfully installed latest transformers
2023-07-26 11:49:52.937 | INFO     | autotrain.cli.run_setup:run:40 - Installing latest peft@main
2023-07-26 11:50:06.325 | INFO     | autotrain.cli.run_setup:run:42 - Successfully installed latest peft
2023-07-26 11:50:06.325 | INFO     | autotrain.cli.run_setup:run:46 - Installing latest diffusers@main
2023-07-26 11:50:23.825 | INFO     | autotrain.cli.run_setup:run:48 - Successfully installed la

In [ ]:
!autotrain llm -help

2023-07-26 11:50:39.489189: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
usage: autotrain <command> [<args>] llm
       [-h]
       [--train]
       [--deploy]
       [--inference]
       [--data_path DATA_PATH]
       [--train_split TRAIN_SPLIT]
       [--valid_split VALID_SPLIT]
       [--text_column TEXT_COLUMN]
       [--model MODEL]
       [--learning_rate LEARNING_RATE]
       [--num_train_epochs NUM_TRAIN_EPOCHS]
       [--train_ba

# **Training using AutoTrain**

Command for training

In [ ]:
!autotrain llm --train --deploy --project_name Llama2-CodeGen-PEFT-QLora --model meta-llama/Llama-2-7b --data_path . --text_column text --use_peft --use_int4 --lora_r 32 --lora_alpha 64 --lora_dropout 0.1 --learning_rate 2e-4 --fp16 --train_batch_size 4 --num_train_epochs 5 --trainer sft --push_to_hub --repo_id AhmedSSoliman/Llama2-CodeGen-PEFT-QLora

Training 5 epochs

In [ ]:
!autotrain llm --train --deploy --project_name Llama2-CodeGen-PEFT-QLora --model TinyPixel/Llama-2-7B-bf16-sharded --data_path . --text_column text --use_peft --use_int4 --lora_r 32 --lora_alpha 64 --lora_dropout 0.1 --learning_rate 2e-4 --fp16 --train_batch_size 4 --num_train_epochs 5 --trainer sft --push_to_hub --repo_id AhmedSSoliman/Llama2-CodeGen-PEFT-QLora

[codecarbon INFO @ 13:16:52] Energy consumed for RAM : 0.003854 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 13:16:52] Energy consumed for all GPUs : 0.012621 kWh. Total GPU Power : 31.543000000000003 W
[codecarbon INFO @ 13:16:52] Energy consumed for all CPUs : 0.017175 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:16:52] 0.033651 kWh of electricity used since the beginning.


2023-07-26 13:16:57.000042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
> INFO    Running LLM
> INFO    Train: True
> INFO    loading dataset from csv
Using pad_token, but it is not set yet.

[codecarbon INFO @ 13:17:07] Energy consumed for RAM : 0.003894 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 13:17:07] Energy consumed for all GPUs : 0.012752 kWh. Total GPU Power : 31.543000000000003 W
[codecarbon INFO @ 13:17:07] Energy consumed for all CPUs : 0.017352 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:17:07] 0.033999 kWh of electricity used since the beginning.


[codecarbon INFO @ 13:17:22] Energy consumed for RAM : 0.003934 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 13:17:22] Energy consumed for all GPUs : 0.012884 kWh. Total GPU Power : 31.641000000000002 W
[codecarbon INFO @ 13:17:22] Energy consumed for all CPUs : 0.017529 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:17:22] 0.034347 kWh of electricity used since the beginning.


[codecarbon INFO @ 13:17:37] Energy consumed for RAM : 0.003973 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 13:17:37] Energy consumed for all GPUs : 0.013016 kWh. Total GPU Power : 31.641000000000002 W
[codecarbon INFO @ 13:17:37] Energy consumed for all CPUs : 0.017706 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:17:37] 0.034696 kWh of electricity used since the beginning.


...





pytorch_model-00001-of-00002.bin:  92% 9.21G/9.98G [06:44<00:32, 23.8MB/s]



pytorch_model-00001-of-00002.bin:  92% 9.21G/9.98G [06:45<00:31, 24.2MB/s]



pytorch_model-00001-of-00002.bin:  92% 9.22G/9.98G [06:45<00:27, 27.8MB/s]



pytorch_model-00001-of-00002.bin:  92% 9.22G/9.98G [06:45<00:39, 19.1MB/s]



pytorch_model-00001-of-00002.bin:  92% 9.22G/9.98G [06:45<00:39, 19.3MB/s]



pytorch_model-00001-of-00002.bin:  92% 9.23G/9.98G [06:45<00:35, 21.3MB/s]



pytorch_model-00001-of-00002.bin:  93% 9.23G/9.98G [06:45<00:30, 24.7MB/s]



pytorch_model-00001-of-00002.bin:  93% 9.23G/9.98G [06:46<00:36, 20.1MB/s]



pytorch_model-00001-of-00002.bin:  93% 9.24G/9.98G [06:46<00:35, 20.9MB/s]



pytorch_model-00001-of-00002.bin:  93% 9.24G/9.98G [06:46<00:32, 22.7MB/s]



pytorch_model-00001-of-00002.bin:  93% 9.25G/9.98G [06:46<00:27, 26.2MB/s]



pytorch_model-00001-of-00002.bin:  93% 9.25G/9.98G [06:46<00:37, 19.1MB/s]



pytorch_model-00001-of-00002.bin:  93% 9.25G/9.98G [06:47<00

[codecarbon INFO @ 16:05:07] Energy consumed for RAM : 0.030584 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 16:05:07] Energy consumed for all GPUs : 0.197670 kWh. Total GPU Power : 27.867000000000004 W
[codecarbon INFO @ 16:05:07] Energy consumed for all CPUs : 0.136326 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 16:05:07] 0.364581 kWh of electricity used since the beginning.






pytorch_model-00001-of-00002.bin:  95% 9.48G/9.98G [07:00<00:29, 16.8MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.48G/9.98G [07:00<00:29, 16.7MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.49G/9.98G [07:00<00:27, 17.8MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.49G/9.98G [07:00<00:35, 13.7MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.49G/9.98G [07:00<00:26, 18.3MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.50G/9.98G [07:01<00:27, 17.4MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.50G/9.98G [07:01<00:27, 17.3MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.50G/9.98G [07:01<00:26, 18.2MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.50G/9.98G [07:01<00:32, 14.4MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.51G/9.98G [07:01<00:23, 19.6MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.51G/9.98G [07:01<00:25, 18.2MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.52G/9.98G [07:02<00:25, 17.8MB/s]



pytorch_model-00001-of-00002.bin:  95% 9.52G/9.98G [07:02<00

[codecarbon INFO @ 16:05:22] Energy consumed for RAM : 0.030624 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 16:05:22] Energy consumed for all GPUs : 0.197787 kWh. Total GPU Power : 28.064000000000004 W
[codecarbon INFO @ 16:05:22] Energy consumed for all CPUs : 0.136503 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 16:05:22] 0.364915 kWh of electricity used since the beginning.






pytorch_model-00001-of-00002.bin:  98% 9.80G/9.98G [07:14<00:06, 28.4MB/s]



pytorch_model-00001-of-00002.bin:  98% 9.81G/9.98G [07:15<00:08, 20.8MB/s]



pytorch_model-00001-of-00002.bin:  98% 9.81G/9.98G [07:15<00:06, 24.8MB/s]



pytorch_model-00001-of-00002.bin:  98% 9.82G/9.98G [07:15<00:05, 28.9MB/s]



pytorch_model-00001-of-00002.bin:  98% 9.82G/9.98G [07:15<00:05, 27.8MB/s]



pytorch_model-00001-of-00002.bin:  98% 9.82G/9.98G [07:15<00:06, 22.9MB/s]



pytorch_model-00001-of-00002.bin:  99% 9.83G/9.98G [07:15<00:05, 26.6MB/s]



pytorch_model-00001-of-00002.bin:  99% 9.83G/9.98G [07:15<00:04, 30.7MB/s]



pytorch_model-00001-of-00002.bin:  99% 9.84G/9.98G [07:16<00:04, 28.5MB/s]



pytorch_model-00001-of-00002.bin:  99% 9.84G/9.98G [07:16<00:06, 20.5MB/s]



pytorch_model-00001-of-00002.bin:  99% 9.84G/9.98G [07:16<00:05, 24.5MB/s]



pytorch_model-00001-of-00002.bin:  99% 9.85G/9.98G [07:16<00:04, 27.8MB/s]



pytorch_model-00001-of-00002.bin:  99% 9.85G/9.98G [07:16<00

Training 10 epochs

In [ ]:
!autotrain llm --train --deploy --project_name Llama2-CodeGeneration-PEFT-QLora --model TinyPixel/Llama-2-7B-bf16-sharded --data_path . --use_peft --use_int4 --lora_r 16 --lora_alpha 32 --lora_dropout 0.05 --learning_rate 2e-4 --fp16 --train_batch_size 4 --num_train_epochs 10 --trainer sft --push_to_hub --repo_id AhmedSSoliman/Llama2-CodeGeneration-PEFT-QLora

2023-07-25 07:56:31.168236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-25 07:56:32.519 | INFO     | autotrain.cli.run_llm:run:446 - Running LLM
2023-07-25 07:56:32.519 | INFO     | autotrain.cli.run_llm:run:447 - Train: True
2023-07-25 07:56:32.519 | INFO     | autotrain.trainers.clm:train:38 - loading dataset from csv
Using pad_token, but it is not set yet.
Loading checkpoint shards: 100% 14/14 [01:18<00:00,  5.60s/it]
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Running tokenizer on train dataset:   0% 0/2000 [00:00<?, ? examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1910 > 1024). Running this sequence through the model will result in indexing errors
2023-07-25 07:58:10.484 | INFO 

# **Inference**

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "AhmedSSoliman/Llama2-CodeGen-PEFT-QLora"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True, return_dict=True, load_in_4bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

[codecarbon INFO @ 18:10:52] Energy consumed for RAM : 0.050571 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 18:10:52] Energy consumed for all GPUs : 0.257863 kWh. Total GPU Power : 31.740000000000006 W
[codecarbon INFO @ 18:10:52] Energy consumed for all CPUs : 0.225388 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:10:52] 0.533822 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:11:07] Energy consumed for RAM : 0.050611 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 18:11:07] Energy consumed for all GPUs : 0.257995 kWh. Total GPU Power : 31.740000000000006 W
[codecarbon INFO @ 18:11:07] Energy consumed for all CPUs : 0.225565 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:11:07] 0.534171 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:11:22] Energy consumed for RAM : 0.050650 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 18:11:22] Energy consumed for all GPUs : 0.258127 kWh. Total GPU Power : 31.740000000000006

[codecarbon INFO @ 18:12:37] Energy consumed for RAM : 0.050849 kWh. RAM Power : 9.54426097869873 W
[codecarbon INFO @ 18:12:37] Energy consumed for all GPUs : 0.258789 kWh. Total GPU Power : 31.758000000000003 W
[codecarbon INFO @ 18:12:37] Energy consumed for all CPUs : 0.226627 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:12:37] 0.536265 kWh of electricity used since the beginning.


In [ ]:
#text = """Compute anomaly scores for the time series."""
text = """Write a prgram that add two variables
"""

In [ ]:
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Write a prgram that add two variables

    >>> add_two_variables(1, 2)
    3
### Response: 
def add_two_variables(x, y):
    
    return x + y ### Input: 
Return


# **Inference**

In [ ]:


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


def create_prompt(instruction):
  system = "You are using the Llam2-CodeGen model, a coding assistant that will help the user to resolve the following instruction:\n"
  instruction = "### Input: " + instruction
  return system + "\n" + instruction + "\n\n" + "### Response:" + "\n"

def generate(
        instruction,
        max_new_tokens=128,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        **kwargs,
):
    prompt = create_prompt(instruction)
    print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    #input_ids = inputs["input_ids"].to("cuda")
    #attention_mask = inputs["attention_mask"].to("cuda")

    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            #input_ids=input_ids,
            #attention_mask=attention_mask,
            **inputs,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
            early_stopping=True
        )



    generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    stop_output = "### Input"
    gen_response = (generated_response.split(stop_output))[0]


    #s = generation_output.sequences[0]
    #output = tokenizer.decode(s, skip_special_tokens=True)
    #stop_output = "### Input"

    #gen_response = (output.split(stop_output))[0]


    #return output.split("### Response:")[1].lstrip("\n")
    return gen_response





In [ ]:
instruction = """
 Write a python code for the name Ahmed to be in a reversed order
"""
print(generate(instruction))

You are using the Llam2-CodeGen model, a coding assistant that will help the user to resolve the following instruction:

### Input: 
 Write a python code for the name Ahmed to be in a reversed order


### Response:

Write a python code for the name Ahmed to be in a reversed order.Ћ
def reverse_name(name):
    return name[::-1] 


In [ ]:
instruction = """
Write a program that add five numbers
"""
print(generate(instruction))

You are a coding assistant that will help the user to resolve the following instruction:
### Input: 
Write a program that add five numbers


### Response:

def main():
    
    n1 = int(input("Enter the first number: "))
    n2 = int(input("Enter the second number: "))
    n3 = int(input("Enter the third number: "))
    n4 = int(input("Enter the fourth number: "))
    n5 = int(input("Enter the fifth number: "))

    sum = n1 + n2 + n3 + n4 + n5

    print("The sum of the five numbers is " + str(sum))

    return main<s>


In [ ]:

instruction = """
Write a python code for reading multiple images from path
"""
print(generate(instruction))

You are a coding assistant that will help the user to resolve the following instruction:
### Input: 
Write a python code for reading multiple images from path


### Response:

def read_multiple_images(path):
    
    images = []
    for image_name in os.listdir(path):
        image_path = os.path.join(path, image_name)
        if os.path.isfile(image_path):
            images.append(cv2.imread(image_path))
    return np.array(images)<s> ### Input: 
Returns a list of all the files in the given directory.



In [ ]:
instrution = """
Set width of output ('auto' will auto-detect terminal width)
"""
print(generate(instruction))

You are a coding assistant that will help the user to resolve the following instruction:
### Input: 
Write a python code for reading multiple images from path


### Response:

def read_multiple_images(path):
    
    images = []
    for image_name in os.listdir(path):
        image_path = os.path.join(path, image_name)
        if os.path.isfile(image_path):
            images.append(cv2.imread(image_path))
    return np.array(images)<s> ### Input: 
Returns a list of all the files in the given directory.



In [ ]:
instruction = """
 Write a python code for the name Ahmed to be in a reversed order
"""
print(generate(instruction))